In [1]:
import pathlib
import numpy as np

from aiida import orm, load_profile
from aiida.engine import submit

from qe_tools import CONSTANTS

from aiida_quantumespresso_hp.workflows.hubbard import SelfConsistentHubbardWorkChain
from aiida_quantumespresso.data.hubbard_structure import HubbardStructureData

load_profile()

Profile<uuid='4fe00b7a32994cfca8b6cf43c2d22a53' name='jgeiger'>

In [3]:
! verdi daemon start 4

Starting the daemon with 4 workers... RUNNING


# BaTiO3
Let's define the BTO structure to use as example for this tutorial.

In [8]:
cell = [
    [5.34198, -3.08419, 4.38073],
    [0.00000,  6.16839, 4.38073],
    [-5.3419, -3.08419, 4.38073],
]
cell = CONSTANTS.bohr_to_ang*np.array(cell)

structure = orm.StructureData(cell=cell)

structure.append_atom(position=np.dot([0.48674, 0.4867, 0.4867], cell),  symbols='Ti')
structure.append_atom(position=np.dot([0.51163, 0.5116, 0.0192], cell),  symbols='O' )
structure.append_atom(position=np.dot([0.51163, 0.0192, 0.5116], cell),  symbols='O' )
structure.append_atom(position=np.dot([0.01929, 0.5116, 0.5116], cell),  symbols='O' )
structure.append_atom(position=np.dot([0.99899, 0.9989, 0.9989], cell),  symbols='Ba')

## HubbardStructureData initialization
Let's initialize the HubbardStructureData with the BTO structure!

> Important: You only need to do these steps once! Later just keep track of this testing structure via the PK and reload it from the database so you don't get 100 BaTiO3 strucures in your storage. ^^

In [9]:
hubbard_data = HubbardStructureData(structure=structure)

In [10]:
hubbard_data

<HubbardStructureData: uuid: 121783bc-8579-4b3a-986b-2740cc20fec0 (unstored)>

## Initializing the on-site Hubbard
Let's initialize the on-site Hubbard parameter for the titanium atom.

In [11]:
hubbard_data.initialize_onsites_hubbard('Ti', '3d', 5.0) # index 0 for Ti ==> Hubbard U

Here how it is stored in the class.

In [12]:
hubbard_data.hubbard_parameters

[[0, '3d', 0, '3d', 5.0, [0, 0, 0], 'dudarev']]

## Initializing the inter-site Hubbard
Let's initialize the inter-site Hubbard parameter for the titanium and oxygen atoms.

In [13]:
hubbard_data.initialize_intersites_hubbard('Ti', '3d', 'O', '2p', 0.0001, number_of_neighbours=7)

The parameters are saved in the property `hubbard_parameters` as a list.

In [14]:
hubbard_data.hubbard_parameters

[[0, '3d', 0, '3d', 5.0, [0, 0, 0], 'dudarev'],
 [0, '3d', 3, '2p', 0.0001, [0, 0, 0], 'dudarev'],
 [0, '3d', 1, '2p', 0.0001, [0, 0, 0], 'dudarev'],
 [0, '3d', 2, '2p', 0.0001, [0, 0, 0], 'dudarev'],
 [0, '3d', 1, '2p', 0.0001, [0, 0, 1], 'dudarev'],
 [0, '3d', 2, '2p', 0.0001, [0, 1, 0], 'dudarev'],
 [0, '3d', 3, '2p', 0.0001, [1, 0, 0], 'dudarev'],
 [0, '3d', 3, '2p', 0.0001, [0, 0, -1], 'dudarev']]

In [16]:
print(hubbard_data.get_quantum_espresso_hubbard_card())

HUBBARD	ortho-atomic
 U	Ti-3d	5.0
 V	Ti-3d	O-2p	1	4	0.0001
 V	Ti-3d	O-2p	1	2	0.0001
 V	Ti-3d	O-2p	1	3	0.0001
 V	Ti-3d	O-2p	1	72	0.0001
 V	Ti-3d	O-2p	1	83	0.0001
 V	Ti-3d	O-2p	1	114	0.0001
 V	Ti-3d	O-2p	1	69	0.0001



In [17]:
hubbard_data.store()

<HubbardStructureData: uuid: 121783bc-8579-4b3a-986b-2740cc20fec0 (pk: 2502)>

In [ ]:
hubbard_data.pk

In [10]:
%%bash
# verdi data show 2184

## Running the `SelfConsistentHubbardWorkChain`

In [19]:
hubbard_data = orm.load_node(2502)  # I load the node from the database instead of always regenerating it
# type(hubbard_data)
# from aiida_quantumespresso.data.hubbard_structure import HubbardStructureData

In [20]:
pw_code = orm.load_code('qe-dev-pw@lumi-small')
hp_code = orm.load_code('qe-dev-hp@lumi-small')

builder = SelfConsistentHubbardWorkChain.get_builder_from_protocol(
    pw_code=pw_code,
    hp_code=hp_code,
    hubbard_structure=hubbard_data,
    protocol='fast',
    overrides=pathlib.Path('hubbard_overrides.yaml')
)

builder.skip_first_relax = True

In [21]:
second_submit = submit(builder)
# print(second_submit)

In [19]:
first_submit_pk = 2234
second_submit_pk = 2448

In [7]:
%%bash
verdi process show 2448
printf '=%.0s' {1..100}
echo
verdi process status 2448
printf '=%.0s' {1..100}
echo
verdi process report 2448
printf '=%.0s' {1..100}
echo

Property     Value
-----------  ---------------------------------------------------------------------
type         SelfConsistentHubbardWorkChain
state        Finished [402] The PwRelaxWorkChain sub process failed in iteration 2
pk           2448
uuid         767c3be7-8042-4e79-9fe6-8ec2af85655e
label
description
ctime        2023-02-21 14:30:56.614402+01:00
mtime        2023-02-21 15:22:09.913790+01:00

Inputs                               PK    Type
-----------------------------------  ----  --------------------
hubbard
    hp
        code                         2183  InstalledCode
        qpoints                      2437  KpointsData
        parameters                   2438  Dict
        settings                     2439  Dict
    clean_workdir                    2440  Bool
    max_iterations                   2441  Int
    parallelize_atoms                2442  Bool
relax
    base
        pw
            pseudos
                Ti                   14    UpfData
                O

In [14]:
%%bash
verdi process show 2234
verdi process status 2234
verdi process report 2234

Property     Value
-----------  ---------------------------------------------------------------------
type         SelfConsistentHubbardWorkChain
state        Finished [402] The PwRelaxWorkChain sub process failed in iteration 2
pk           2234
uuid         8a7af431-03dd-44f4-a84c-a17e55ead812
label
description
ctime        2023-02-16 10:03:47.493781+01:00
mtime        2023-02-16 10:21:15.871575+01:00

Inputs                               PK    Type
-----------------------------------  ----  --------------------
hubbard
    hp
        code                         2183  InstalledCode
        qpoints                      2223  KpointsData
        parameters                   2224  Dict
        settings                     2225  Dict
    clean_workdir                    2226  Bool
    max_iterations                   2227  Int
    parallelize_atoms                2228  Bool
relax
    base
        pw
            pseudos
                Ti                   14    UpfData
                O

bash: line 2: repchar: command not found


In [6]:
%%bash
verdi process report 2313

2023-02-16 10:18:52 [52 | REPORT]: [2313|PwBaseWorkChain|run_process]: launching PwCalculation<2318> iteration #1
2023-02-16 10:21:15 [57 | REPORT]: [2313|PwBaseWorkChain|report_error_handled]: PwCalculation<2318> failed with exit status 305: Both the stdout and XML output files could not be read or parsed.
2023-02-16 10:21:15 [58 | REPORT]: [2313|PwBaseWorkChain|report_error_handled]: Action taken: unrecoverable error, aborting...
2023-02-16 10:21:15 [59 | REPORT]: [2313|PwBaseWorkChain|inspect_process]: PwCalculation<2318> failed but a handler detected an unrecoverable problem, aborting
2023-02-16 10:21:15 [60 | REPORT]: [2313|PwBaseWorkChain|on_terminated]: remote folders will not be cleaned


In [11]:
!verdi process report 2310

2023-02-16 10:18:51 [51 | REPORT]: [2310|PwRelaxWorkChain|run_relax]: launching PwBaseWorkChain<2313>
2023-02-16 10:18:52 [52 | REPORT]:   [2313|PwBaseWorkChain|run_process]: launching PwCalculation<2318> iteration #1
2023-02-16 10:21:15 [57 | REPORT]:   [2313|PwBaseWorkChain|report_error_handled]: PwCalculation<2318> failed with exit status 305: Both the stdout and XML output files could not be read or parsed.
2023-02-16 10:21:15 [58 | REPORT]:   [2313|PwBaseWorkChain|report_error_handled]: Action taken: unrecoverable error, aborting...
2023-02-16 10:21:15 [59 | REPORT]:   [2313|PwBaseWorkChain|inspect_process]: PwCalculation<2318> failed but a handler detected an unrecoverable problem, aborting
2023-02-16 10:21:15 [60 | REPORT]:   [2313|PwBaseWorkChain|on_terminated]: remote folders will not be cleaned
2023-02-16 10:21:15 [61 | REPORT]: [2310|PwRelaxWorkChain|inspect_relax]: relax PwBaseWorkChain failed with exit status 300
2023-02-16 10:21:15 [62 | REPORT]: [2310|PwRelaxWorkChain|on

In [12]:
!verdi process report 2313

2023-02-16 10:18:52 [52 | REPORT]: [2313|PwBaseWorkChain|run_process]: launching PwCalculation<2318> iteration #1
2023-02-16 10:21:15 [57 | REPORT]: [2313|PwBaseWorkChain|report_error_handled]: PwCalculation<2318> failed with exit status 305: Both the stdout and XML output files could not be read or parsed.
2023-02-16 10:21:15 [58 | REPORT]: [2313|PwBaseWorkChain|report_error_handled]: Action taken: unrecoverable error, aborting...
2023-02-16 10:21:15 [59 | REPORT]: [2313|PwBaseWorkChain|inspect_process]: PwCalculation<2318> failed but a handler detected an unrecoverable problem, aborting
2023-02-16 10:21:15 [60 | REPORT]: [2313|PwBaseWorkChain|on_terminated]: remote folders will not be cleaned


In [6]:
!verdi process report 2234

2023-02-16 10:03:50 [18 | REPORT]: [2234|SelfConsistentHubbardWorkChain|setup]: system is treated to be non-magnetic because `nspin == 1` in `scf.pw.parameters` input.
2023-02-16 10:03:51 [19 | REPORT]: [2234|SelfConsistentHubbardWorkChain|should_run_relax]: skip_first_relax has been set True; skipping first relaxion...
2023-02-16 10:03:51 [20 | REPORT]: [2234|SelfConsistentHubbardWorkChain|run_scf_smearing]: launching PwBaseWorkChain<2236> with smeared occupations
2023-02-16 10:03:54 [21 | REPORT]:   [2236|PwBaseWorkChain|run_process]: launching PwCalculation<2241> iteration #1
2023-02-16 10:06:21 [26 | REPORT]:   [2236|PwBaseWorkChain|results]: work chain completed after 1 iterations
2023-02-16 10:06:21 [27 | REPORT]:   [2236|PwBaseWorkChain|on_terminated]: remote folders will not be cleaned
2023-02-16 10:06:21 [28 | REPORT]: [2234|SelfConsistentHubbardWorkChain|recon_scf]: after relaxation, system is determined to be an insulator
2023-02-16 10:06:22 [29 | REPORT]: [2234|SelfConsiste

SelfConsistentHubbardWorkChain<2234> Finished [402] [1:while_(should_run_iteration)(1:if_(should_run_relax)(1:inspect_relax))]
    ├── PwBaseWorkChain<2236> Finished [0] [4:results]
    │   ├── create_kpoints_from_distance<2237> Finished [0]
    │   └── PwCalculation<2241> Finished [0]
    ├── PwBaseWorkChain<2261> Finished [0] [4:results]
    │   ├── create_kpoints_from_distance<2262> Finished [0]
    │   └── PwCalculation<2266> Finished [0]
    ├── HpWorkChain<2284> Finished [0] [2:results]
    │   └── HpBaseWorkChain<2286> Finished [0] [3:results]
    │       └── HpCalculation<2288> Finished [0]
    ├── structure_relabel_kinds<2307> Finished [0]
    └── PwRelaxWorkChain<2310> Finished [401] [1:while_(should_run_relax)(1:inspect_relax)]
        └── PwBaseWorkChain<2313> Finished [300] [3:while_(should_run_process)(2:inspect_process)]
            ├── create_kpoints_from_distance<2314> Finished [0]
            └── PwCalculation<2318> Finished [305]


In [9]:
%%bash
verdi process report 2318
# verdi process report 2310


*** 2318: None
*** (empty scheduler output file)
*** Scheduler errors:

Lmod is automatically replacing "cce/14.0.2" with "gcc/11.2.0".


Lmod is automatically replacing "PrgEnv-cray/8.3.3" with "PrgEnv-gnu/8.3.3".


Due to MODULEPATH changes, the following have been reloaded:
  1) cray-mpich/8.1.18


Lmod is automatically replacing "craype-x86-rome" with "craype-x86-milan".

MPICH ERROR [Rank 33] [job id 2901031.0] [Thu Feb 16 11:20:10 2023] [nid002232] - Abort(1) (rank 33 in comm 0): application called MPI_Abort(MPI_COMM_WORLD, 1) - process 33

aborting job:
application called MPI_Abort(MPI_COMM_WORLD, 1) - process 33
MPICH ERROR [Rank 60] [job id 2901031.0] [Thu Feb 16 11:20:10 2023] [nid002232] - Abort(1) (rank 60 in comm 0): application called MPI_Abort(MPI_COMM_WORLD, 1) - process 60

aborting job:
application called MPI_Abort(MPI_COMM_WORLD, 1) - process 60
MPICH ERROR [Rank 110] [job id 2901031.0] [Thu Feb 16 11:20:10 2023] [nid002232] - Abort(1) (rank 110 in comm 0): applicati

In [ ]:
show_columns_sampling = ['li_number', 'formula', 'space_group', 'lithiation', 'li_mpc', 'cell_mpc', 'sg_mpc']

pd.set_option('display.max_rows', 10)
for sampling_df_key in sampling_df_keys[:2]:
    current_df = sampling_df_dict[sampling_df_key][show_columns_sampling]
    print("Structure: {}, Size: {}, # Configs: {}".format(*sampling_df_key, current_df.shape[0]))
    current_df
    # view(sampling_df_dict[sampling_df_key]['ase_structure'].values)
    # break

Structure: olivine_LiFePO, Size: 1, # Configs: 10


,li_number,formula,space_group,lithiation,li_mpc,cell_mpc,sg_mpc
0,4,Fe4Li4O16P4,Pm,1.00,1,10,4
1,3,Fe4Li3O16P4,P1,0.75,2,10,6
2,3,Fe4Li3O16P4,P1,0.75,2,10,6
3,2,Fe4Li2O16P4,P1,0.50,4,10,6
4,2,Fe4Li2O16P4,P1,0.50,4,10,6
5,2,Fe4Li2O16P4,Pm,0.50,4,10,4
6,2,Fe4Li2O16P4,Pm,0.50,4,10,4
7,1,Fe4LiO16P4,P1,0.25,2,10,6
8,1,Fe4LiO16P4,P1,0.25,2,10,6
9,0,Fe4O16P4,Pm,0.00,1,10,4


Structure: olivine_LiFePO, Size: 2, # Configs: 462


,li_number,formula,space_group,lithiation,li_mpc,cell_mpc,sg_mpc
0,7,Fe8Li7O32P8,P1,0.875,14,66,378
1,7,Fe8Li7O32P8,P1,0.875,14,66,378
2,6,Fe8Li6O32P8,P1,0.750,56,66,378
3,6,Fe8Li6O32P8,P1,0.750,56,66,378
4,6,Fe8Li6O32P8,Pm,0.750,56,66,30
...,...,...,...,...,...,...,...
457,2,Fe8Li2O32P8,Pc,0.250,56,66,18
458,2,Fe8Li2O32P8,Pm,0.250,56,66,30
459,2,Fe8Li2O32P8,Pc,0.250,56,66,18
460,1,Fe8LiO32P8,P1,0.125,14,66,378
